# Final Project

Name: Kartikeya Sharma <br/>Class: CSCI 349 - Intro to Data Mining  
Semester: Spring 2021  
Instructor: Brian King  

### Step 0: Enivornment Setup

#### Step 0.0: Installations

- **phonenumbers:** Python wrapper for Google's phone numbers API: ```conda install phonenumbers```<br/>
- **us:** package for conveniently working with state abbreviations: ```pip install us``` because conda-forge installation does not work (and is not listed as a primary way of installing this package, so we're stuck with pip)<br/>
- **uszipcode:** package that has a vast amount of information on zipcodes, including the major city, post office city, common city, county, state, **area code list** (helpful), **latitude** (helpful), **longitude** (helpful), timezone, demographics (population, population_density, population by age, population by gender, population by race, etc.)... talk about data mining ```pip install uszipcode``` (pip had a solid installation of this)

#### Step 0.1: Imports

In [150]:
%time
# imports used in the course
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# from  matplotlib.ticker import PercentFormatter
# from sklearn.model_selection import train_test_split, KFold
# from sklearn.utils import shuffle
# from sklearn.metrics import classification_report, confusion_matrix, f1_score
# from sklearn.model_selection import cross_validate, cross_val_predict
# from sklearn.model_selection import GridSearchCV

# from sklearn.preprocessing import StandardScaler
# from sklearn.tree import DecisionTreeClassifier

# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import Input, Model
# from tensorflow.keras.layers import Dense, Activation
# from tensorflow.keras.optimizers import Adam, SGD
# from scikeras.wrappers import KerasClassifier

CPU times: user 1e+03 ns, sys: 1 µs, total: 2 µs
Wall time: 4.05 µs


In [151]:
# custom imports (not used in the course)
import phonenumbers
import us
from uszipcode import SearchEngine # importing what is needed per the documentation

```uszipcode``` documentation<sup>5</sup>

In [152]:
%time 
try:
    df_raw = pd.read_csv("../data/Consumer_Complaints_Data_-_Unwanted_Calls_raw.csv")
except:
    # import from online if not available on machine
    df_raw = pd.read_csv('https://query.data.world/s/24xzbr2jaeuohhwmrzhj7jcyrdamlw')

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.05 µs


In [153]:
df = df_raw.sample(frac=(0.95/116.4)).copy(deep=True)

```df``` will be considered the sample of the data set, where ```df_raw``` will be considered as the whole data set; this is for simplicity and convenience by design, so I don't have to write ```df_samp``` or ```df_sample``` every time. 

I am choosing 0.95/116.4 because ~116.4 MB is how much all of the data takes up (whole raw data set) and ~0.95 (MB) - ends up a bit more than that, though - is the amount of space that we want the sample (also before preprocessing) to take, keeping the data being worked with under 1 MB; this is the fraction of data that will be retained for analysis.

<br/>Data provided from fcc.gov<sup>2</sup>

### Step 1: Pre-Processing

#### Step 1.0 Inspect Data at a Macro Level

The purposes of this analysis, None will be np.NaN.

In [154]:
df.fillna(value=np.nan, inplace=True)

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9582 entries, 254700 to 638445
Data columns (total 12 columns):
 #   Column                                   Non-Null Count  Dtype 
---  ------                                   --------------  ----- 
 0   Ticket ID                                9582 non-null   int64 
 1   Date of Issue                            9528 non-null   object
 2   Time of Issue                            8280 non-null   object
 3   Form                                     9582 non-null   object
 4   Method                                   9554 non-null   object
 5   Issue                                    9582 non-null   object
 6   Caller ID Number                         8857 non-null   object
 7   Type of Call or Messge                   8309 non-null   object
 8   Advertiser Business Number               7434 non-null   object
 9   State                                    9578 non-null   object
 10  Zip                                      9582 non-nul

In [156]:
df.head()

,Ticket ID,Date of Issue,Time of Issue,Form,Method,Issue,Caller ID Number,Type of Call or Messge,Advertiser Business Number,State,Zip,Location (Center point of the Zip Code)
254700,721355,12/03/2015,5:00 pm,Phone,Wired,Robocalls,916-313-4201,Abandoned Calls,NaN,FL,33407,"FL 33407\n(26.753216, -80.080411)"
891545,2704729,08/14/2018,10:00 am,Phone,Wireless (cell phone/other mobile device),Unwanted Calls,906-369-7786,Prerecorded Voice,None,MI,49920,"MI 49920\n(46.171662, -88.344524)"
783830,2436732,05/09/2018,10:32 a.m.,Phone,Wireless (cell phone/other mobile device),Unwanted Calls,None,Prerecorded Voice,972-589-8469,TX,75075,"TX 75075\n(33.022357, -96.738333)"
519806,1232747,09/19/2016,6:38 p.m.,Phone,Wired,Robocalls,608-416-1654,Abandoned Calls,NaN,WI,53235,"WI 53235-4322\n(42.96978, -87.875065)"
61800,3663225,11/15/2019,NaN,Phone,Wired,Unwanted Calls,None,NaN,None,IL,60453,"IL 60453\n(41.715086, -87.753789)"


#### Step 1.1: Discard, Format, Downsize Each Column

Column #0 "Ticket ID": Unless there are some duplicate ticket values that should be looked into further, I intend on discarding them.<sup>2</sup>

In [157]:
df["Ticket ID"].value_counts().sort_values(ascending=False).head(1)

985089    1
Name: Ticket ID, dtype: int64

Clearly no duplicate values. Discarding Column #0 "Ticket ID."

In [158]:
df.drop(columns="Ticket ID", inplace=True)

In [ ]:
def _convert_to_EST_or_nan(ts_str: str):
    try:
        ts = pd.to_datetime(ts_str)
        ts = ts.tz_convert("US/Eastern")
        ts = ts.floor('Min')
        return ts
    except:
        return pd.NaT
    
df["Ticket Created"] = df["Ticket Created"].apply(_convert_to_EST_or_nan)

In [ ]:
df = df[df["Ticket Created"].notnull()]

In [ ]:
df["Ticket Created"].head()

In [ ]:
df["Ticket Created"].isna().sum() / len(df.index)

<br/>Column #2 "Date of Issue": This column represents the date when the user actually experienced the issue. It will be stored as a date without a time zone (since not all observations gave a particular time so that the dates can be informatively localized accordingly).

What % are NaN values? Just to get an idea (ideally not more than a percent or two).

In [ ]:
df["Date of Issue"].isna().sum() / len(df.index)

In [ ]:
def _convert_datetime(dt):
    try:
        return pd.to_datetime(dt)
    except: 
        return np.NaN

In [ ]:
df["Date of Issue"] = df["Date of Issue"].apply(_convert_datetime)

In [ ]:
df = df[df["Date of Issue"] >= pd.to_datetime("October 31, 2014")]

Starting point of the data set is October 31, 2014.<sup>2</sup>

In [ ]:
df["Date of Issue"].isna().sum() / len(df.index)

<br/>Column #3 "Time of Issue": This column represents the time when the user actually experienced the issue. This information appears to be too specific for our analysis.<sup>2</sup>

What % are NaN values? Just to get an idea (ideally not more than a percent or two).

In [ ]:
df["Time of Issue"].isna().sum() / len(df.index)

A lot of non-values and not a *necessary* column (could miss out on analyzing what time of data the calls tend to come in, and by time zone, that is, but there is too much missing data to do this). Dropping it from the DataFrame.

In [ ]:
df.drop(columns="Time of Issue", inplace=True)

<br/>Column #4 "Form": This represents the method through which the consumers were contacted by the reported caller.<sup>2</sup>

In [ ]:
df["Form"].value_counts()

These are all "Phone"; we don't need it. (The whole raw data set of the sample that we are analyzing is, in turn, part of a larger data set maintained by the FCC with complaints regarding phone calls, TV service providers, Internet service providers, etc.<sup>8</sup> 

In [ ]:
df.drop(columns="Form", inplace=True)

<br/>Column #5 "Method": How the reported caller contacted the consumer.<sup>2</sup>

What % are NaN values? Just to get an idea (ideally not more than a percent or two).

In [ ]:
df["Method"].isna().sum() / len(df.index)

In [ ]:
df["Method"] = pd.Categorical(df["Method"], ordered=False)

In [ ]:
df["Method"].head()

There are three possibilities (nominal) categories: Internet (VOIP), which is something like Google Voice, wired, which is a landline, and wireless, which is a cell phone.<sup>2</sup>

It is not necessary to clarify that wireless means cell phone or other mobile device. This category will be renamed to wireless.

In [ ]:
df["Method"].cat.rename_categories(
    new_categories = {"Wireless (cell phone/other mobile device)": "Wireless"},
    inplace=True
)

In [ ]:
df["Method"].isna().sum() / len(df.index)

<br/>Column #6 "Issue": 

What % are NaN values? Just to get an idea (ideally not more than a percent or two).

In [ ]:
df["Issue"].isna().sum() / len(df.index)

In [ ]:
df["Issue"] = pd.Categorical(df["Issue"], ordered=False)

In [ ]:
df["Issue"].head()

Looking close at the FCC descriptions of the different attributes, robocalls and telemarketing are currently being marked as unwanted calls instead of those two categories; hence, the values in this column are not consistent throughout across the reports/over time. This column will be removed, and the data set will be considered as containing data that represents reports of 'unwanted/spam' calls.<sup>2</sup>

In [ ]:
df.drop(columns="Issue", inplace=True)

<br/>Column #7 "Caller ID Number": number (of reported call from corresponding unwanted caller) that appeared on the consumer's caller ID<sup>2</sup>

We will use the Python wrapper of google's phonenumbers library.<sup>3</sup>

Per "Please enter the phone number in the following format 555-555-5555" instructions on the FCC complaint form, all numbers are assumed to be USA, which makes sense since the FCC is a USA regulatory body which has jurisdiction in the USA.<sup>4</sup>

What % are NaN values? Just to get an idea (ideally not more than a percent or two or 5% at the most).

In [ ]:
df["Caller ID Number"].isna().sum() / len(df.index)

Even though there is quite a bit missing, it might be valuable to keep observations with "Caller ID Number" as np.NaN (private/unknown caller or consumer deleted spam message/incoming call entry and does not have the number now)?

In [ ]:
def _parse_ph_number(ph_num: str):
    try:
        ph_num = str(ph_num)
        ph_num_parsed = phonenumbers.parse(ph_num, "US")
    except phonenumbers.NumberParseException: 
        return np.NaN
    if not phonenumbers.is_possible_number(ph_num_parsed):
        return np.NaN
    return ph_num_parsed

In [ ]:
# to not print the slew of lines that comes out with phonenumbers methods
# for whatever reason (there are some patches that need to be resolved with
# wrapping the Google Java library in Python)
df["Caller ID Number"] = \
df["Caller ID Number"].apply(_parse_ph_number);

In [ ]:
df["Caller ID Number"].head()

In [ ]:
df["Caller ID Number"].isna().sum() / len(df.index)

About a 5%-point increase on average in NaN values after preprocessing this column.

<br/>Column #8 "Type of Call or Messge": The type of the call (or message) received. Live voice? Prerecorded message? Text message?

What % are NaN values? Just to get an idea (ideally not more than a percent or two or 5% at the most).

In [ ]:
df["Type of Call or Messge"].isna().sum() / len(df.index)

In [ ]:
df["Type of Call or Messge"] = pd.Categorical(df["Type of Call or Messge"], ordered=False)

In [ ]:
df["Type of Call or Messge"].head()

<br/>Column #9 "Advertiser Business Number": The number of the advertiser that the caller claims to be associated with.

What % are NaN values? Just to get an idea (ideally not more than a percent or two or 5% at the most).

In [ ]:
df["Advertiser Business Number"].isna().sum() / len(df.index)

Way too many missing values. Deleting this column.

In [ ]:
df.drop(columns="Advertiser Business Number", inplace=True)

<br/>Column #10 "State": State in which the reporting consumer resides in<sup>2</sup>

What % are NaN values? Just to get an idea (ideally not more than a percent or two or 5% at the most).

In [ ]:
df["State"].isna().sum() / len(df.index)

Not many relatively (percentage-wise) missing values (< 0.1% on average). Nice!

In [ ]:
def _validate_state(state_ab: str):
    try:
        if us.states.lookup(state_ab) in us.states.STATES:
            return state_ab
        # including DC but leaving out Puerto Rico (not that many entries
        # proportionally anyway)
        elif us.states.lookup(state_ab) is us.states.DC:
            return state_ab
        else:
            return np.NaN
    except: # if np.NaN, for example
        return np.NaN

```us``` documentation<sup>6</sup>

In [ ]:
df["State"] = df["State"].apply(_validate_state)

We don't have to eliminate observations without valid state abbreviations. Perhaps we can extrapolate the state of the customer from their area code, i.e. the phone number that received the call.

In [ ]:
df["State"] = pd.Categorical(df["State"], ordered=False)

State abbreviations are values within a nominal variable.

In [ ]:
df["State"].head()

We have each of the 50 states covered in our data sample. Yay!

In [ ]:
df["State"].isna().sum() / len(df.index)

Still, not many relatively (percentage-wise) missing values (< 0.1% on average).

<br/>Column #11 "Zip": The zip code of where the consumer resides<sup>2</sup>

What % are NaN values? Just to get an idea (ideally not more than a percent or two or 5% at the most).

In [ ]:
df["Zip"].isna().sum() / len(df.index)

None, if any, are missing (on average).

Because we don't need all of the vast amount that this package provides *at the moment*, given that the latitude, longitude, state information is already given in our data set, we will start by using the simple version of the backend database provided by the ```uszipcode``` package.

In [ ]:
search = SearchEngine(simple_zipcode=True, db_file_dir="../data/zip_data_raw")

In [ ]:
def _validate_and_get_info_zipcode(zip: str):
    try:
        zip_obj = search.by_zipcode(zip)
        if zip_obj.zipcode is None:
            return np.NaN
        return zip_obj
    except:
        return np.NaN

In [ ]:
df["Zip Info"] = df["Zip"].apply(_validate_and_get_info_zipcode)

How many are NaN after preprocessing (we'll still remove them right after). Checking this adds another check to the preprocessing of this column (to ensure that the preprocessing algorithm/method above is not simply failing and inserting unwaranted NaNs all over).

In [ ]:
df["Zip"].isna().sum() / len(df.index)

None, if any, on average. Great.

We will now remove all entries with invalid ZIP Codes. We will be using the ZIP Code objects, which are now stored in the "Zip Info" column (newly created column), to also store otherr information associated with the Zip Code, such as latitude and longitude information; this way, the information is consolidated within the SimpleZipcode object. Plus, we can avoid painfully parsing "Location..." column (next column), keep the data consistent within the uszipcode package (information all from one database), and verify the validity of the latitude/longitude data all at the same time through the SimpleZipcode object.

In [ ]:
df = df[df["Zip Info"]!=np.NaN]

A zip code is technically a nominal variable (even though there may be *many* of them). We can perhaps sort the data sample by a zip code category to understand which zip codes had the most reports, for instance. Making the Zip Info column into a Categorical may not make sense if the pd.Categorical method does not pick up on two different objects potentially being the same if their contents are the same. For simplicity, the Zip Info column will not be converted into a pd.Categorical.

In [ ]:
df["Zip"] = pd.Categorical(df["Zip"], ordered=False)

In [ ]:
df["Zip"].head()

```uszipcode``` documentation<sup>5</sup>

<br/>Column #12 "Location (Center point of the Zip Code)": center of the zip code, not by consumer's address or anything specifically (data set maintains consumer privacy, which is the ethical thing to do and is solid computer science research ethics)<sup>2</sup>

Per the explanation above, I am removing this column.<br/>
*"We will be using the ZIP Code objects, which are now stored in the "Zip Info" column (newly created column), to also store otherr information associated with the Zip Code, such as latitude and longitude information; this way, the information is consolidated within the SimpleZipcode object. Plus, we can avoid painfully parsing "Location..." column (next column), keep the data consistent within the uszipcode package (information all from one database), and verify the validity of the latitude/longitude data all at the same time through the SimpleZipcode object."*

In [ ]:
df.drop(columns="Location (Center point of the Zip Code)", inplace=True)

<br/>All in all (after the column conversions):

In [ ]:
df.head()

### Step 2: EDA

#### Step 2.0: Plotting General Distributional Data

In [ ]:
freq_date_of_issue = df["Date of Issue"].groupby(
    by=[df["Date of Issue"].dt.year, df["Date of Issue"].dt.month]
).count()
freq_date_of_issue.index.rename(names=["Year", "Month"], inplace=True)
df_freq_date_of_issue = pd.DataFrame(freq_date_of_issue)
df_freq_date_of_issue.reset_index(inplace=True)
df_freq_date_of_issue.rename({"Date of Issue": "Frequency"}, axis=1, inplace=True)

display(df_freq_date_of_issue)

```freq_date_of_issue``` setup inspired by Stack Overflow contributor<sup>7</sup>

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))
sns.boxplot(data=df_freq_date_of_issue, x="Frequency", ax=ax)
fig.suptitle("Distribution of Frequencies in a (Month, Year)");

In [ ]:
df_freq_date_of_issue.describe()

In [ ]:
fg = sns.FacetGrid(data=df_freq_date_of_issue, col="Year", sharey=False)
fg.map(sns.histplot, "Frequency", kde=True, binwidth=)

In [ ]:
sns.displot(kind="hist", data=freq_date_of_issue, kde=True)

In [ ]:
fg = sns.catplot(kind="count", x=df_freq_date_of_issue, height=5, aspect=2)

### References
1. Data from https://data.world/kgarrett/unwanted-calls
2. Verified description of the variable using https://opendata.fcc.gov/Consumer/Consumer-Complaints-Data-Unwanted-Calls/vakf-fz8e
3. Python wrapper of Google's ```phonenumbers``` library (documentation): https://pypi.org/project/phonenumbers/
4. FCC unwanted call complaint form: https://consumercomplaints.fcc.gov/hc/en-us/requests/new?ticket_form_id=39744
5. ```uszipcode``` documentation: https://pypi.org/project/uszipcode/
6. ```us``` documentation: https://pypi.org/project/us/
7. Stack Overflow: https://stackoverflow.com/questions/27365467/can-pandas-plot-a-histogram-of-dates
8. FCC https://consumercomplaints.fcc.gov/hc/en-us